In [6]:
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path.append(os.path.abspath(os.path.join(".")))

import pandas as pd

from icu_pipeline.pipeline import (
    Pipeline,
    DataSource,
    SourceConfig,
)

PG_PASS = os.getenv("PG_PASS")
PG_USER = os.getenv("PG_USER")
PG_HOST = os.getenv("PG_HOST")
PG_PORT = os.getenv("PG_PORT")

MIMIC_DB = os.getenv("PG_MIMIC_NAME")
EICU_DB = os.getenv("PG_EICU_NAME")
AMDS_DB = os.getenv("PG_AMDS_NAME")
SIC_DB = os.getenv("PG_SIC_NAME")
HIRID_DB = os.getenv("PG_HIRID_NAME")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: No module named 'icu_pipeline'

In [2]:
mimic_config = SourceConfig(
    f"postgresql+psycopg://{PG_USER}:{PG_PASS}@{PG_HOST}/{MIMIC_DB}",
    limit=10,
    chunksize=10
)
eicu_config = SourceConfig(
    f"postgresql+psycopg://{PG_USER}:{PG_PASS}@{PG_HOST}/{EICU_DB}",
    limit=10,
    chunksize=10
)
amds_config = SourceConfig(
    f"postgresql+psycopg://{PG_USER}:{PG_PASS}@{PG_HOST}/{AMDS_DB}",
    limit=10,
    chunksize=10
)
sic_config = SourceConfig(
    f"postgresql+psycopg://{PG_USER}:{PG_PASS}@{PG_HOST}/{SIC_DB}",
    limit=10,
    chunksize=10
)
hirid_config = SourceConfig(
    f"postgresql+psycopg://{PG_USER}:{PG_PASS}@{PG_HOST}/{HIRID_DB}",
    limit=10,
    chunksize=10
)

NameError: name 'SourceConfig' is not defined

In [6]:
from icu_pipeline.sink.pandas import PandasSink

pipeline = Pipeline(
    source_configs={
        DataSource.MIMICIV: mimic_config,
    },
    sink_mapper=PandasSink(),
)

concepts = ["HeartRate"]

In [7]:
result = pipeline.transform(concepts)
print(pipeline._graph)

Pipe Class: InMemoryPipe
Node Class: InMemoryNode


In [8]:
result = [{
    k: v for k,v in c.items()
} for c in result]
out = {}
for c in pipeline._sink_mapper._sources:
    out[c] = pd.concat([v[c] for v in result])
out[name]

FileNotFoundError: [Errno 2] No such file or directory: 'conceptbase/concepts/HeartRate.yml'

In [16]:
out["Vancomycine_dosage"].iloc[0]

KeyError: 'Vancomycine_dosage'

In [ ]:
print(pipeline._graph)

In [ ]:
pipeline._sink_mapper._sources

In [ ]:
out["HeartRate"]